# 6. Поиск текста

# 6.1 Поиск с помощью LIKE

Выберите из таблицы **users** всех пользователей, чьи фамилии начинаются на А без учета регистра.

Отсортируйте данные в алфавитном порядке по фамилии и имени.

**Решение:**

```MySQL
SELECT * FROM users
WHERE last_name LIKE "А%"
ORDER BY last_name, first_name ASC;
```

Выберите из таблицы **domains** имена всех доменов в зоне .ru и отсортируйте их по дате создания.

Выводить нужно только имена доменов.

**Решение:**

```MySQL
SELECT domain FROM domains
WHERE domain LIKE '%.ru'
ORDER BY created ASC;
```

Выберите из таблицы **domains** имена всех доменов, у которых домен первого уровня состоит из 3 символов.

Отсортируйте данные по имени домена в алфавитном порядке.

В итоговой таблице должен остаться только столбец domain.

**Решение:**

```MySQL
SELECT domain FROM domains
WHERE domain LIKE '%.___'
ORDER BY domain ASC;
```

Один из участников ДТП скрылся с места аварии.

Однако прохожие успели заметить часть номера автомобиля нарушителя.

В частности они запомнили, что серия заканчивается на ОР (русскими буквами), а регистрационный номер начинается с единицы. Также они запомнили марку автомобиля — Ford, и его цвет — желтый или зеленый (мнения разделились, так как на улице было уже темно).

Получите из таблицы **cars** все автомобили, которые подходят под описание.

**Решение:**

```MySQL
SELECT * FROM cars
WHERE number LIKE '_1__ОР%' AND mark = 'Ford' AND (color = 'желтый' OR color = 'зеленый');
```

В таблице **wines** содержится список вин с ценами. Отдел маркетинга посчитал, что цены, которые заканчиваются на два нуля, следует уменьшить на 1 рубль, чтобы в конце было 99 (400 -> 399).

Так цена будет казаться меньше. Напишите SQL запрос для решения этой задачи.

**Решение:**

```MySQL
UPDATE wines
SET price = price - 1 WHERE price LIKE '%00';
```

```MySQL
UPDATE wines
SET price = price - 1  WHERE price % 100 = 0;
```

Правильное название марки автомобилей — **BMW** (все буквы заглавные).

Однако в таблице много автомобилей BMW, которые записаны неверно.
Найдите все автомобили с неверным названием.

**Решение:**

```MySQL
SELECT * FROM cars
WHERE mark LIKE 'BMW' AND mark NOT LIKE BINARY 'BMW';
```

# 6.2 Полнотекстовый поиск

```MySQL
CREATE FULLTEXT INDEX ind_name ON products(name);
```

В таблице **products** содержатся товары из женского магазина. Пользователь ищет джинсы фирмы Mango 36 или 38 размера. Получите из таблицы id, название и цену всех подходящих товаров. Учитывайте, что некоторых позиций на складе нет.

Таблица была создана с помощью следующего запроса:

```MySQL
CREATE TABLE products (
    id INT UNSIGNED NOT NULL PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100) NOT NULL,
    count SMALLINT UNSIGNED NOT NULL DEFAULT 0,
    price INTEGER UNSIGNED NOT NULL DEFAULT 0,
    sizes SET('32','34','36','38','40','42','44','46','48','50','52','M','L','S','XL','XS','2XL','4XL') NULL,
    FULLTEXT INDEX name(name)
);
```

**Решение:**

```MySQL
SELECT id, name, price FROM products
WHERE MATCH (name) AGAINST ('+Джинсы +Mango' IN BOOLEAN MODE)
AND (FIND_IN_SET (36, sizes) OR FIND_IN_SET (38, sizes))
AND count > 0;
```

```MySQL
SELECT id, name, price FROM products
WHERE MATCH (name) AGAINST ('+Джинсы +Mango' IN BOOLEAN MODE)
AND (sizes LIKE '%36%' OR sizes like '%38%')
AND count > 0;
```

Полнотекстовые индексы можно создавать сразу по нескольким столбцам.
Для этого нужно указать все колонки в скобках через запятую.
Например так можно создать индекс search по названию и описанию:

```MySQL
CREATE FULLTEXT INDEX search ON products(name, description);
```

Так как индекс создается по нескольким столбцам, то при поиске в конструкции MATCH надо перечислить их все:

```MySQL
SELECT * FROM products 
WHERE MATCH(name, description) AGAINST ('платье детское');
```
Найдите в таблице **forum** все посты, которые содержат слова «ошибка» или «проблема».

Выведите только id и название поста.

Таблица была создана с помощью следующего запроса:

```MySQL
CREATE TABLE forum (
    id INT UNSIGNED NOT NULL PRIMARY KEY AUTO_INCREMENT,
    subject VARCHAR(255) NULL,
    author_id INTEGER NULL,
    post TEXT NULL,
    FULLTEXT INDEX forum_text(subject, post)
    );
```
**Решение:**

```MySQL
SELECT id, subject FROM forum
WHERE MATCH (subject, post) AGAINST ('ошибка проблема');
```

В предыдущей задаче мы искали слова «ошибка» или «проблема», однако в текстах эти слова склоняются и мы пропустили «ошибки», «ошибками», «проблемы» и тд.

И хотя полнотекстовый поиск в MySQL не является морфологическим (не учитывает склонения), мы всё же можем обработать и такие ситуации. В логическом режиме в конце слов вместо окончаний можно добавлять звездочку, тогда поиск будет учитывать все варианты написания.
Например, «товар*» будет равносилен словам «товар», «товары», «товарами» и тд.

Измените ваш предыдущий запрос так, чтобы он искал ошибки и проблемы с любыми окончаниями.
Структура и данные в таблице остались такими же.

**Решение:**

```MySQL
SELECT id, subject FROM forum
WHERE MATCH (subject, post) AGAINST ('ошибк* проблем*' IN BOOLEAN MODE);
```

Получите из таблицы **products** все джинсы и жилеты, за исключеним товаров фирмы Mango.

Таблица была создана с помощью следующего запроса:

```MySQL
CREATE TABLE products (
    id INT UNSIGNED NOT NULL PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100) NOT NULL,
    count SMALLINT UNSIGNED NOT NULL DEFAULT 0,
    price INTEGER UNSIGNED NOT NULL DEFAULT 0,
    sizes SET('32','34','36','38','40','42','44','46','48','50','52','M','L','S','XL','XS','2XL','4XL') NULL,
    FULLTEXT INDEX name(name)
);
```
**Решение:**

```MySQL
SELECT * FROM products
WHERE MATCH (name) AGAINST ('джинсы жилет -Mango' IN BOOLEAN MODE);
```

Как вы знаете, полнотекстовый поиск ищет слова по отдельности. Но иногда нужно найти фразу целиком, например если это название книги или вы точно знаете что ищете. Для это достаточно заключить фразу в двойные кавычки:

```MySQL
SELECT * FROM films 
WHERE MATCH(name) AGAINST ('"Кремниевая долина"' IN BOOLEAN MODE);
```

Найдите в таблице **products** все товары, которые содержат фразу «Джинсы Mango».
Данные отсортируйте по цене (по умолчанию данные сортируются по релевантности, но мы можем изменить это поведение).
Выведите только id, название и цену.

Таблица была создана с помощью следующего запроса:

```MySQL
CREATE TABLE products (
    id INT UNSIGNED NOT NULL PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100) NOT NULL,
    count SMALLINT UNSIGNED NOT NULL DEFAULT 0,
    price INTEGER UNSIGNED NOT NULL DEFAULT 0,
    sizes SET('32','34','36','38','40','42','44','46','48','50','52','M','L','S','XL','XS','2XL','4XL') NULL,
    FULLTEXT INDEX name(name)
);
```

**Решение:**

```MySQL
SELECT id, name, price FROM products
WHERE MATCH (name) AGAINST ('"Джинсы Mango"' IN BOOLEAN MODE)
ORDER BY price;
```

Иногда даже сложные выражения внутри AGAINST не позволяют найти нужные записи. Однако вы можете добавить несколько конструкций MATCH...AGAINST в один запрос, разделив их операторами AND или OR:

```MySQL
SELECT * FROM films WHERE
MATCH (name) AGAINST ('+Кремниевая +долина' IN BOOLEAN MODE) OR
MATCH (name) AGAINST ('+Силиконовая +долина' IN BOOLEAN MODE);
```

Получите из таблицы **products** все джинсы и юбки компании Mango.

Выведите только id, название и цену. Данные отсортируйте по цене. Не выводите товары, которых нет в наличии.


**Решение:**

```MySQL
SELECT id, name, price FROM products
WHERE (MATCH (name) AGAINST ('+Джинсы +Mango' IN BOOLEAN MODE)
OR MATCH (name) AGAINST ('+Юбка +Mango' IN BOOLEAN MODE))
AND count > 0
ORDER BY price;
```